# Vector Database Notebook

In [9]:
import json

In [10]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [11]:
documents = []

for course in docs_raw:
    for doc in course['documents']:
        doc['course'] = course['course']
        documents.append(doc)


In [12]:
#! pip install sentence_transformers==2.7.0

## 1. Embedding

Embedding is a vector representation of a text (or other types of data). In such a form application of different ML/DL algorithms become possible.

In [13]:
from sentence_transformers import SentenceTransformer

In [14]:
model = SentenceTransformer('all-mpnet-base-v2')

In [15]:
# dimensionality of our embedding
len(model.encode("This is a simple sentence"))

768

In [16]:
# create a dense vector using pre-trainde model for our answer(aka `text` field)
opertaions = []

for doc in documents:
    doc['text_vector'] = model.encode(doc['text']).tolist()
    opertaions.append(doc)

In [17]:
opertaions[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'text_vector': [-0.03570346534252167,
  -0.0689142644405365,
  -0.04448341578245163,
  0.0062501514330506325,
  -0.04224711284041405,
  -0.006454005837440491,
  0.02754814736545086,
  -0.01861027628183365,
  -0.052037470042705536,
  -0.002404735190793872,
  0.05745450034737587,
  -0.015592597424983978,
  0.015488816425204277,
  0.0987015962600708,
  0.015649540349841118,
  -0.06

## 3. Vector DataBase

Permits **optimal** and **effective** storage and search

### 3.1 Setup ElasticSearch connection

Running ES with Docker
```
docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

In [18]:
from elasticsearch import Elasticsearch

In [19]:
es_client = Elasticsearch('http://localhost:9200')
es_client.info()

ObjectApiResponse({'name': '675ce062f461', 'cluster_name': 'docker-cluster', 'cluster_uuid': '4gR5tzWATWyRv8KHoGRj7Q', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [20]:
health = es_client.cluster.health()
print(health)

{'cluster_name': 'docker-cluster', 'status': 'green', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 2, 'active_shards': 2, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 0, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 100.0}


In [21]:
# Adjust high and low watermarks temporarily
es_client.cluster.put_settings(body={
    "transient": {
        "cluster.routing.allocation.disk.watermark.low": "85%",
        "cluster.routing.allocation.disk.watermark.high": "95%",
        "cluster.routing.allocation.disk.watermark.flood_stage": "98%"
    }
})

ObjectApiResponse({'acknowledged': True, 'persistent': {}, 'transient': {'cluster': {'routing': {'allocation': {'disk': {'watermark': {'low': '85%', 'flood_stage': '98%', 'high': '95%'}}}}}}})

In [22]:
es_client.cluster.health()

ObjectApiResponse({'cluster_name': 'docker-cluster', 'status': 'green', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 2, 'active_shards': 2, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 0, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 100.0})

#### 3.1.1 Create Mappings and Index

* Mapping will define how we gonna store and index our data
* Each document is a collection of fields, which each have their own data type
* Mapping is like a db schema

In [23]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector",
                            "dims": 768,
                            "index": True,
                            "similarity": "cosine"},
        }
    }
}

In [24]:
index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

#### 3.1.2 Add documents into index


In [26]:
for doc in opertaions:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

#### 3.1.3 Create end user query

In [27]:
search_term = "windows or mac?"
vector_search_term = model.encode(search_term)


#### 3.1.4 Perform Semantic Search 

In [28]:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000,
}

In [29]:
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'XKK325AB9Go2v_eaB3fD',
  '_score': 0.71479195,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': 'b6K325AB9Go2v_eaMXpJ',
  '_score': 0.6134734,
  '_source': {'question': 'WSL instructions',
   'course': 'mlops-zoomcamp',
   'section': 'Module 1: Introduction',
   'text': 'If you wish to use WSL on your windows machine, here are the setup instructions:\nCommand: Sudo apt install wget\nGet Anaconda download address here. wget <download address>\nTurn on Docker Desktop WFree Download | AnacondaSL2\nCommand: git clone <github repository address>\nVSCODE on WSL\nJupyter: pip3 install jupyter\nAdded by Gregory Morris (gwm1980@gmail.com)\nAll in all softwares a

#### 3.1.5 Perform Keyword search with Semantic Search (Hybrid/Advanced Search)


If using cosine similarity with normalized vectors, scores will fall btswn 0 and 1.

With advanced search may be different score.=> check the output and use `explain` parameter to see how the score was calculated

In [31]:
# Included "knn" in the search query performs a semantic search along with the filter on a keyword
knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}

In [32]:
response = es_client.search(
    index=index_name,
    query={
        "match": {"section": "General course-related questions"},
    },
    knn=knn_query,
    size=5,
    explain=True # will tell how the score are calculated
)

In [39]:
response["hits"]["hits"][0]

{'_shard': '[course-questions][0]',
 '_node': 'Y1r0IrBpSzmokALJKMnKvg',
 '_index': 'course-questions',
 '_id': 'XKK325AB9Go2v_eaB3fD',
 '_score': 11.614713,
 '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
  'section': 'General course-related questions',
  'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
  'course': 'data-engineering-zoomcamp',
  'text_vector': [-0.026965467259287834,
   -0.0006261487724259496,
   -0.016629496589303017,
   0.05285152047872543,
   0.054765284061431885,
   -0.031339891254901886,
   0.029942581430077553,
   -0.04808562994003296,
   0.04467552527785301,
   0.0058394744992256165,
   0.016233038157224655,
   0.012001179158687592,
   -0.03122226521372795,
   0.016600513830780983,
   -0.04886903986334801,
   -0.06496306508779526,
   0.046434223651885986,
   -0.009297751821577549,
   -0.0642528384923935,
   -0.01373270247131586,
   -0.01597621664404869,
